# Baseline Table Retrieval

Direct query encoding with MiniLM-L6-v2 and FAISS brute-force search.

## Setup

In [1]:
import pandas as pd
import numpy as np
import json
import faiss
from sentence_transformers import SentenceTransformer
from collections import defaultdict
from tqdm import tqdm

/Users/minhrua/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/minhrua/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [2]:
# Load tables
tables_df = pd.read_csv('data/wikitables_mini.csv')
print(f"Loaded {len(tables_df)} tables")
print(f"Columns: {list(tables_df.columns)}")
tables_df.head(2)

Loaded 2932 tables
Columns: ['table_id', 'page_title', 'section_title', 'table_caption', 'headers', 'sample_data']


,table_id,page_title,section_title,table_caption,headers,sample_data
0,table-0001-249,Auburn Tigers swimming and diving,Summer Olympic Games Beijing 2008,Summer Olympic Games Beijing 2008,"[""Athlete"", ""Nation"", ""Total"", ""Gold"", ""Silver...","[[""[Fr\u00e9d\u00e9rick_Bousquet|Fr\u00e9d\u00..."
1,table-0001-400,Bisphenol A,Low-dose exposure in animals,Low-dose exposure in animals,"[""Dose (\u00b5g/kg/day)"", ""[Environmental_Work...","[[""0.025"", ""\""Permanent changes to genital tra..."


## Generate Table Descriptions

Convert tables to natural text: "Table about [caption]. Section: [section]. From [page]. Columns: [headers]. Sample data: [first row]"

Limit: 256 chars (MiniLM-L6-v2 token limit)

In [3]:
def generate_table_description(row, max_total_chars=256):
    """Generate table description as natural, coherent text."""
    parts = []
    
    # Caption
    if pd.notna(row['table_caption']) and str(row['table_caption']).strip():
        parts.append(f"Table about {str(row['table_caption']).strip()}.")
    
    # Section
    if pd.notna(row['section_title']) and str(row['section_title']).strip():
        parts.append(f"Section: {str(row['section_title']).strip()}.")
    
    # Page
    if pd.notna(row['page_title']) and str(row['page_title']).strip():
        parts.append(f"Page: {str(row['page_title']).strip()}.")
    
    # Headers
    try:
        headers = json.loads(row['headers'])
        if headers:
            headers_str = ', '.join([str(h) for h in headers[:10]])
            parts.append(f"Columns: {headers_str}.")
    except:
        pass
    
    # Sample data
    try:
        sample_data = json.loads(row['sample_data'])
        if sample_data and len(sample_data) > 0:
            first_row = ', '.join([str(cell) for cell in sample_data[0][:5]])
            parts.append(f"Sample data: {first_row}.")
    except:
        pass
    
    description = ' '.join(parts)
    return description[:max_total_chars]

# Test
print("Example descriptions:")
print("=" * 80)
for i in range(3):
    desc = generate_table_description(tables_df.iloc[i])
    print(f"\n{i+1}. {desc}")

Example descriptions:

1. Table about Summer Olympic Games Beijing 2008. Section: Summer Olympic Games Beijing 2008. Page: Auburn Tigers swimming and diving. Columns: Athlete, Nation, Total, Gold, Silver, Bronze, Events. Sample data: [Frédérick_Bousquet|Frédérick Bousquet], [France

2. Table about Low-dose exposure in animals. Section: Low-dose exposure in animals. Page: Bisphenol A. Columns: Dose (µg/kg/day), [Environmental_Working_Group|Environmental Working Group], Study Year. Sample data: 0.025, "Permanent changes to genital tract", 

3. Table about Players. Section: Players. Page: Charlotte Bobcats all-time roster. Columns: *, *, [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expansion_Draft|2004 Expansion Draft], [2004_NBA_Expa


In [4]:
# Generate descriptions for all tables
print("Generating descriptions...")
table_descriptions = []
table_ids = []

for idx, row in tqdm(tables_df.iterrows(), total=len(tables_df)):
    table_descriptions.append(generate_table_description(row))
    table_ids.append(row['table_id'])

print(f"Generated {len(table_descriptions)} descriptions")
print(f"Length stats - Mean: {np.mean([len(d) for d in table_descriptions]):.1f}, Max: {max([len(d) for d in table_descriptions])}")

Generating descriptions...


100%|██████████| 2932/2932 [00:00<00:00, 34505.14it/s]

Generated 2932 descriptions
Length stats - Mean: 226.2, Max: 256


## Encode Tables

In [5]:
# Load encoder
model_name = 'all-MiniLM-L6-v2'
print(f"Loading {model_name}...")
encoder = SentenceTransformer(model_name)
print(f"Dimension: {encoder.get_sentence_embedding_dimension()}")

Loading all-MiniLM-L6-v2...
Dimension: 384


In [6]:
# Encode descriptions
print("Encoding...")
table_embeddings = encoder.encode(
    table_descriptions, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)
print(f"Shape: {table_embeddings.shape}")

Encoding...


Batches: 100%|██████████| 92/92 [00:04<00:00, 21.33it/s]

Shape: (2932, 384)


## Build FAISS Index

In [7]:
# Build FAISS index
print("Building index...")
index = faiss.IndexFlatIP(encoder.get_sentence_embedding_dimension())
index.add(table_embeddings.astype('float32'))
print(f"✓ Index built with {index.ntotal} tables")

Building index...
✓ Index built with 2932 tables


### Test Custom Queries

In [8]:
# Test custom queries - change test_query and run
test_query = "swimming techniques for beginners"
top_k = 5

print(f"Query: '{test_query}'")
print("=" * 80)

# Encode and search
test_emb = encoder.encode([test_query], convert_to_numpy=True, normalize_embeddings=True).astype('float32')
scores_test, indices_test = index.search(test_emb, top_k)

for rank in range(top_k):
    idx = indices_test[0][rank]
    table_id = table_ids[idx]
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    
    print(f"\n{rank + 1}. {table_id} (score: {scores_test[0][rank]:.4f})")
    print(f"   Page: {row['page_title']}")
    print(f"   Caption: {row['table_caption']}")
    print(f"   Description: {table_descriptions[idx][:120]}...")
    print("-" * 80)

Query: 'swimming techniques for beginners'

1. table-0562-150 (score: 0.4244)
   Page: Swimming at the 2008 Summer Olympics – Qualification
   Caption: Women's 10 km
   Description: Table about Women's 10 km. Section: Women's 10 km. Page: Swimming at the 2008 Summer Olympics – Qualification. Columns: ...
--------------------------------------------------------------------------------

2. table-0604-492 (score: 0.4241)
   Page: List of Indian records in swimming
   Caption: Men
   Description: Table about Men. Section: Men. Page: List of Indian records in swimming. Columns: Event, Time, , Name, Club, Date, Meet,...
--------------------------------------------------------------------------------

3. table-0379-471 (score: 0.4146)
   Page: Swimming at the 2008 Summer Olympics – Women's 100 metre freestyle
   Caption: Records
   Description: Table about Records. Section: Records. Page: Swimming at the 2008 Summer Olympics – Women's 100 metre freestyle. Columns...
--------------------------

## Evaluation

### Load Queries and Relevance Judgments

In [9]:
# Load queries
queries = {}
with open('data/queries.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(None, 1)
        if len(parts) == 2:
            query_id, query_text = parts
            queries[query_id] = query_text

print(f"Loaded {len(queries)} queries")
print("Examples:", list(queries.items())[:3])

# Load qrels (relevance judgments)
qrels = defaultdict(dict)
with open('data/qrels.txt', 'r') as f:
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 4:
            query_id, table_id, relevance = parts[0], parts[2], int(parts[3])
            qrels[query_id][table_id] = relevance

qrels = dict(qrels)
print(f"Loaded qrels for {len(qrels)} queries")

Loaded 60 queries
Examples: [('1', 'world interest rates table'), ('2', '2008 beijing olympics'), ('3', 'fast cars')]
Loaded qrels for 60 queries


---

### Encode Queries and Retrieve

In [10]:
# Encode queries
query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

print(f"Encoding {len(query_texts)} queries...")
query_embeddings = encoder.encode(
    query_texts, batch_size=32, show_progress_bar=True,
    convert_to_numpy=True, normalize_embeddings=True
)

# Search top 100
k = 100
print(f"Searching top-{k}...")
scores, indices = index.search(query_embeddings.astype('float32'), k)

results = {qid: [table_ids[idx] for idx in indices[i]] for i, qid in enumerate(query_ids)}
print(f"✓ Retrieved {len(results)} query results")

# Show examples
print("\n" + "=" * 80)
print("EXAMPLE RESULTS")
print("=" * 80)
for qid in list(queries.keys())[:3]:
    print(f"\nQuery {qid}: '{queries[qid]}'")
    for rank, tid in enumerate(results[qid][:3], 1):
        rel = qrels.get(qid, {}).get(tid, 0)
        score_val = scores[query_ids.index(qid)][rank-1]
        page = tables_df[tables_df['table_id'] == tid].iloc[0]['page_title']
        print(f"  {rank}. {tid} (score: {score_val:.3f}, rel: {rel}) - {page}")
print("=" * 80)

Encoding 60 queries...


Batches: 100%|██████████| 2/2 [00:00<00:00, 20.79it/s]

Searching top-100...
✓ Retrieved 60 query results

EXAMPLE RESULTS

Query 1: 'world interest rates table'
  1. table-0552-510 (score: 0.631, rel: 1) - Single deposit
  2. table-0730-168 (score: 0.602, rel: 0) - List of Renminbi exchange rates
  3. table-0552-511 (score: 0.592, rel: 0) - Single deposit

Query 2: '2008 beijing olympics'
  1. table-0001-249 (score: 0.636, rel: 0) - Auburn Tigers swimming and diving
  2. table-0511-959 (score: 0.618, rel: 0) - Scottish Olympic medallists
  3. table-1578-197 (score: 0.613, rel: 2) - China at the Olympics

Query 3: 'fast cars'
  1. table-1435-370 (score: 0.502, rel: 0) - Cars (soundtrack)
  2. table-0005-922 (score: 0.497, rel: 1) - Fast Cars and Superstars: The Gillette Young Guns Celebrity Race
  3. table-0990-862 (score: 0.491, rel: 1) - Speed Dreams


In [11]:
### Calculate Metrics

# Evaluation functions
def recall_at_k(retrieved, relevant, k):
    if len(relevant) == 0:
        return 0.0
    retrieved_at_k = set(retrieved[:k])
    return len(retrieved_at_k & relevant) / len(relevant)

def ndcg_at_k(retrieved, relevance, k):
    if len(relevance) == 0:
        return 0.0
    dcg = sum(relevance.get(retrieved[i], 0) / np.log2(i + 2) for i in range(min(k, len(retrieved))))
    ideal_rels = sorted(relevance.values(), reverse=True)[:k]
    idcg = sum(rel / np.log2(i + 2) for i, rel in enumerate(ideal_rels))
    return dcg / idcg if idcg > 0 else 0.0

In [12]:
# Evaluate
k_values = [1, 5, 10, 20]
metrics = defaultdict(list)

for query_id, retrieved in results.items():
    if query_id not in qrels:
        continue
    relevance = qrels[query_id]
    relevant = set(tid for tid, rel in relevance.items() if rel > 0)
    
    for k in k_values:
        metrics[f'Recall@{k}'].append(recall_at_k(retrieved, relevant, k))
        metrics[f'nDCG@{k}'].append(ndcg_at_k(retrieved, relevance, k))

# Print results
print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)
print("\nRecall:")
for k in k_values:
    print(f"  Recall@{k:2d}: {np.mean(metrics[f'Recall@{k}']):.4f}")
print("\nnDCG:")
for k in k_values:
    print(f"  nDCG@{k:2d}  : {np.mean(metrics[f'nDCG@{k}']):.4f}")
print("="*60)


EVALUATION RESULTS

Recall:
  Recall@ 1: 0.0748
  Recall@ 5: 0.2714
  Recall@10: 0.4013
  Recall@20: 0.5677

nDCG:
  nDCG@ 1  : 0.4667
  nDCG@ 5  : 0.5048
  nDCG@10  : 0.5241
  nDCG@20  : 0.5675


## Inspect Results

In [13]:
# Inspect specific query results
query_id = '1'

print(f"Query {query_id}: {queries[query_id]}")
print("="*80)

for i, table_id in enumerate(results[query_id][:5], 1):
    row = tables_df[tables_df['table_id'] == table_id].iloc[0]
    rel = qrels.get(query_id, {}).get(table_id, 0)
    score_val = scores[query_ids.index(query_id)][i-1]
    
    print(f"\n{i}. {table_id} (score: {score_val:.4f}, relevance: {rel})")
    print(f"   Page: {row['page_title']}")
    print(f"   Section: {row['section_title']}")
    print(f"   Caption: {row['table_caption']}")
    
    try:
        headers = json.loads(row['headers'])
        print(f"   Headers: {headers[:5]}{'...' if len(headers) > 5 else ''}")
    except:
        pass
    
    try:
        sample = json.loads(row['sample_data'])
        print(f"   Sample ({len(sample)} rows): {sample[0][:3]}...")
    except:
        pass
    
    print(f"   Description: {table_descriptions[table_ids.index(table_id)]}")
    print("-" * 80)

Query 1: world interest rates table

1. table-0552-510 (score: 0.6308, relevance: 1)
   Page: Single deposit
   Section: Real World Example
   Caption: Real World Example
   Headers: ['Amount', 'Period', '[Interest_rate|Interest Rate]', 'Compounding']
   Sample (1 rows): ['5,000.00 USD', '4 Year', '3.50%']...
   Description: Table about Real World Example. Section: Real World Example. Page: Single deposit. Columns: Amount, Period, [Interest_rate|Interest Rate], Compounding. Sample data: 5,000.00 USD, 4 Year, 3.50%, Annually.
--------------------------------------------------------------------------------

2. table-0730-168 (score: 0.6021, relevance: 0)
   Page: List of Renminbi exchange rates
   Section: List of World Bank nominal exchange rates
   Caption: List of World Bank nominal exchange rates
   Headers: ['World Bank annual average middle exchange rate for US dollar to Chinese yuan ( 1 US dollar to Chinese yuan )', 'World Bank annual average middle exchange rate for US dollar to 